# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [10]:
#!jupyter nbextension enable --py gmaps
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key
# configure gmaps
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
# read CSV file as DataFrame
weather_df = pd.read_csv('cities.csv').drop(columns=['Unnamed: 0'])
weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country
0,constitucion,-35.33,-72.42,59.32,78,0,7.00,CL
1,kaitangata,-46.28,169.85,53.69,62,5,8.37,NZ
2,port elizabeth,-33.92,25.57,57.20,87,100,4.70,ZA
3,onda,39.97,-0.25,59.00,79,0,3.69,ES
4,presidencia roque saenz pena,-26.79,-60.44,61.30,54,0,6.11,AR
...,...,...,...,...,...,...,...,...
538,fram,-26.98,-55.88,56.41,68,0,3.62,PY
539,kamenka,51.32,42.77,52.00,76,100,3.00,RU
540,severnoye,56.35,78.36,67.71,53,82,5.99,RU
541,murgab,37.50,61.97,70.29,42,90,19.75,TM


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
# Create location coordinates to plot on my map

heat_locations = weather_df[['Lat', 'Lng']]

# Create map figure with coordinates
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(heat_locations, 
                                 dissipating=False,
                                 max_intensity=400, 
                                 weights = weather_df['Humidity'],
                                 point_radius= 5)

# Add Layers to map figure
fig.add_layer(heat_layer)                     
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
# Keep only rows that meet the following criteria:
# 1) A max temperature lower than 80 degrees but higher than 70.
# 2) Wind speed less than 10 mph.
# 3) Zero cloudiness.
ideal_weather = weather_df.loc[\
                               (weather_df['Max Temp']>70) &\
                               (weather_df['Max Temp']< 80) &\
                               (weather_df['Cloudiness']== 0)&\
                               (weather_df['Windspeed']< 10),:].dropna()

ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country
19,piacabucu,-10.41,-36.43,77.38,85,0,6.08,BR
118,puerto ayora,-0.74,-90.35,75.99,92,0,4.41,EC
173,aswan,24.09,32.91,78.71,35,0,1.72,EG
202,namibe,-15.20,12.15,76.10,50,0,3.71,AO
234,sao jose da coroa grande,-8.90,-35.15,77.63,85,0,4.34,BR
294,sao felix do xingu,-6.64,-51.99,74.44,84,0,2.10,BR
376,atar,20.52,-13.05,76.23,21,0,8.66,MR
419,huaihua,27.55,109.96,79.59,66,0,6.82,CN
489,riberalta,-10.98,-66.10,74.68,74,0,1.77,BO
501,bardiyah,31.76,25.09,72.84,69,0,7.11,LY


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
# Set up proper parameters to pass on to itterrows
params = {
    "location": '',
    "types": 'hotel',
    "keyword": 'hotel',
    "radius": 5000,
    "key": g_key}

# Itterrows to loop through DataFrame's Lat, Lng, and City columns
for index, row in ideal_weather.iterrows():
    ideal_lat = row['Lat']
    ideal_lng =  row['Lng']
    ideal_city = row['City']
    # Pass on proper info to params
    params['location'] = f'{str(ideal_lat)},{str(ideal_lng)}'
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Read URL with json
    response = requests.get(base_url, params).json()
    # On error, keep going
    try:
        # Grab necessary information from json data
        hotel = response['results'][0]['name']
        # Place data into DataFrame
        ideal_weather.loc[index,'Hotel Name']=hotel
    # Skip these possible issues
    except IndexError:
        print(f'Could not find hotel for {ideal_city}')

In [15]:
# Drop rows with missing info
hotel_df = ideal_weather.dropna()
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Hotel Name
19,piacabucu,-10.41,-36.43,77.38,85,0,6.08,BR,Pousada Raizes
118,puerto ayora,-0.74,-90.35,75.99,92,0,4.41,EC,Hotel Solymar Galapagos
173,aswan,24.09,32.91,78.71,35,0,1.72,EG,Sofitel Legend Old Cataract Aswan
202,namibe,-15.20,12.15,76.10,50,0,3.71,AO,Hotel ibis Styles Iu Namibe
234,sao jose da coroa grande,-8.90,-35.15,77.63,85,0,4.34,BR,Pousada Vivenda Oriente
294,sao felix do xingu,-6.64,-51.99,74.44,84,0,2.10,BR,Hotel Serrano
376,atar,20.52,-13.05,76.23,21,0,8.66,MR,Odar kanawal
419,huaihua,27.55,109.96,79.59,66,0,6.82,CN,Royal Santo Exquisite Hotel
489,riberalta,-10.98,-66.10,74.68,74,0,1.77,BO,Pacifico Apart Hotel
501,bardiyah,31.76,25.09,72.84,69,0,7.11,LY,Hotel Burdy


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
                             info_box_content=hotel_info)

fig.add_layer(markers)                     
fig


Figure(layout=FigureLayout(height='420px'))